<a href="https://colab.research.google.com/github/lucianosilva-github/logicanddiscretemathematics/blob/main/LOGIC%2BDISCRETEMATH_CLASS_05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div class="alert alert-block alert-info">

#**CLASS 05 - LINEAR TEMPORAL LOGIC (LTL) - PART III**
**Learning Objectives:**
*   Reviewing of LTL Operators (unary + binary operators)
*   Kripke Structures
*   Model Checking in PL (Propositional Logic) and LTL
*  Implementation of Model Checking with PL and LTL
</div>


## **LTL OPERATORS REVIEW**

A PL (Propositional Logic) formula is either:

* ⊤ (**true**) or ⊥ (**false**)
* a propositional variable
* ¬𝜑1, 𝜑1∧𝜑2, 𝜑1∨𝜑2, or 𝜑1→𝜑2 where both 𝜑1 and 𝜑2 are PL formulas

LTL (Linear Temporal Logic) formulas have the form 𝐴𝜌 where 𝜌 is a LTL path formula and a LTL path formula is either:

* ⊤ or ⊥
*  propositional variable
*  ¬𝜑1, 𝜑1∧𝜑2, 𝜑1∨𝜑2, or 𝜑1→𝜑2, where both 𝜑1 and 𝜑2 are LTL path formulas
* 𝐗𝜑1, 𝐅𝜑1, 𝐆𝜑1, 𝜑1𝐔𝜑2, or 𝜑1𝐑𝜑2 where both 𝜑1 and 𝜑2 are LTL path formulas.


**KRIPKE STRUCTURES**

A Kripke structure is a **directed graph**, equipped with a set of initial nodes, such that **every node** is source of some edge and it **is labeled by a set of atomic propositions**. The nodes of Kripke structure are called states:

A Kripke structure is a tuple (𝑆,𝑆0,𝑅,𝐿) such that:

* 𝑆 is a finite set of states
* 𝑆0⊆𝑆 is a set of initial states
* 𝑅⊆𝑆×𝑆 is a set of transitions such that for all 𝑠∈𝑆 there exists a (𝑠,𝑠′)∈𝑅 for some 𝑠′∈𝑆
* 𝐿:𝑆→2^{AP} maps each state into a set of atomic propositions. Sometimes, the set of initial states is omitted. In such cases, 𝑆 and 𝑆0 coincide.

A computation of a Kripke structure (𝑆,𝑆0,𝑅,𝐿) is an infinite path of (𝑆,𝑅) that starts from some 𝑠∈𝑆0. In the following code we are specifying a Kripke structure using the Python module pyModelChecking.

In [2]:
!pip3 install pyModelChecking


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip


- S: conjunto de estados => Programa apresenta 3 estados ("3 linhas")
- S0: estado inicial => onde o programa começa a funcionar
- R: conjunto de relações => partindo do estado 0 chega no estado 1 e do estado 1 chega ao estado 2 e depois para no estado 2: (2, 2) (poderia ser um loop também)

In [3]:
from pyModelChecking import Kripke

K=Kripke(S=[0,1,2], S0=[0], R=[(0,1),(1,2),(2,2)], L={0: set(['p']), 1:set(['p','q']), 2: set([''])})

In [4]:
x = 3 # ESTADO 0 | p="x igual a 3" ou x_igual_a_3
y = x + 1 # ESTADO 1 q="y == 4" ou q="y > x" ou q="x < y"

K = Kripke(
    S=[0, 1], 
    S0=[0],
    R=[(0, 1), (1, 1)],
    L={0: set(["x_igual_a_3"]), 1: set(["x_igual_a_3", "x_igual_a_4"])}
)

# set -> serve para falar que naquele estado do programa vale x coisa

**EXERCISE 1**

Interpret the Kripke Structure K:

Type your interpretation here





**EXERCISE 2** 

Implement a Kripke Structure for the following Python program:

x=3

y=x+1


In [5]:
#TYPE YOUR IMPLEMENTATION HERE

K = Kripke(
    S=[0, 1], 
    S0=[0],
    R=[(0, 1), (1, 1)],
    L={0: set(["x_igual_a_3"]), 1: set(["x_igual_a_3", "y_igual_a_4"])}
    # no estado 1 poderia ser x_igual_a_5
)

**EXERCISE 3** 

Implement a Kripke Structure for the following Python program:

x=5

if x>0:

    y=1;

else:

    y=0

In [6]:
#TYPE YOUR IMPLEMENTATION HERE

x = 5
if x > 0:
    y = 1
else:
    y = 0

# estado 3 serve para poder referenciar um estado onde não tem nada => para poder parar a execução
K = Kripke(
    S=[0, 1, 2], 
    S0=[0],
    R=[(0, 1), (0, 2), (1, 3), (2, 3), (3, 3)],
    L={0: set(["x_igual_a_5"]), 1: set(["x_maior_que_0", "y_igual_a_1"]), 2: set(["x_menor_igual_que_0", "y_igual_a_0"])}
    # no estado 1 poderia ser x_igual_a_5
)

**EXERCISE 4** 

Implement a Kripke Structure for the following Python program:

x=5

while x>0:

    x=x-1

In [7]:
#TYPE YOUR IMPLEMENTATION HERE

**SEMANTICS**

If 𝐾 is a Kripke structure, 𝑠 one of its states, and 𝜑 a propositional formula, we write 𝐾,𝑠⊨𝜑 (to be read “𝐾 and 𝑠 satisfy 𝜑 ”) meaning that 𝜑 holds at state 𝑠 in 𝐾.

Let 𝐾 be the Kripke structure (𝑆,𝑆0,𝑅,𝐿); the relation ⊨ is defined recursively as follows:

* 𝐾,𝑠⊨⊤ and 𝐾,𝑠⊭⊥ for any state 𝑠∈𝑆
* if 𝑝∈𝐴𝑃, then 𝐾,𝑠⊨𝑝 ⟺ 𝑝∈𝐿(𝑠)
* 𝐾,𝑠⊨¬𝜑 ⟺ 𝐾,𝑠⊭𝜑
* 𝐾,𝑠⊨𝜑1∧𝜑2 ⟺ 𝐾,𝑠⊨𝜑1 and 𝐾,𝑠⊨𝜑2
* 𝐾,𝑠⊨𝜑1∨𝜑2 ⟺ 𝐾,𝑠⊨𝜑1 or 𝐾,𝑠⊨𝜑2
* 𝐾,𝑠⊨𝜑1→𝜑2 ⟺ 𝐾,𝑠⊭𝜑1 or 𝐾,𝑠⊨𝜑2

**MODEL CHECKING**

Model checking is a technique to establish the set of states in Kripke structure that satisfy a given temporal formula. More formally, provided a Kripke structure 𝐾=(𝑆,𝑆0,𝑅,𝐿) and a temporal formula 𝜑, model checking aims to identify 𝑆′⊆𝑆 such that 𝐾,𝑠𝑖⊨𝜑 for all 𝑠𝑖∈𝑆′.


In [8]:
from pyModelChecking import *
from pyModelChecking.LTL import *

K=Kripke(R=[(0,0),(0,1),(1,2),(2,2),(3,3)], L={0: set(['p']), 1:set(['p','q']),3:set(['p'])})
modelcheck(K,'A (p)')

{0, 1, 3}

In [9]:
modelcheck(K,'A (q)')

{1}

In [10]:
modelcheck(K,'A (not q)')

{0, 2, 3}

In [11]:
modelcheck(K,'A (p & q)')

{1}

**EXERCISE 5** 

Implement a Kripke Structure for the following Python program and check some formulas:

x=3

y=x+1

In [12]:
#TYPE YOUR IMPLEMENTATION HERE

**EXERCISE 6** 

Implement a Kripke Structure for the following Python program and check some formulas:
```
0: x=5

1: if x>0:

2:    y=1;

3: else:

4:    y=0
```

In [26]:
#TYPE YOUR IMPLEMENTATION HERE

K = Kripke(
    S=[0, 1, 2, 3, 4], 
    S0=[0],
    R=[(0, 1), (1, 2), (2, 2), (0, 3), (3, 4), (4, 4)],
    L={0: set(["x_igual_a_5"]), 1: set(["x_igual_a_5"]), 2: set(["x_igual_a_0", "y_igual_a_0"]), 3: set(["x_igual_a_0"]), 4: set(["x_igual_a_0", "y_igual_a_0"])}
)

**EXERCISE 7** 

Implement a Kripke Structure for the following Python program and check some formulas:

x=5

while x>0:

    x=x-1

In [19]:
#TYPE YOUR IMPLEMENTATION HERE

K = Kripke(
    S=[0, 1, 2], 
    S0=[0],
    R=[(0, 1), (0, 2), (1, 3), (2, 3), (3, 3)],
    L={0: set(["x_igual_a_5"]), 1: set(["x_maior_que_0", "y_igual_a_1"]), 2: set(["x_menor_igual_que_0", "y_igual_a_0"])}
    # no estado 1 poderia ser x_igual_a_5
)

### **HOMEWORK** 

**EXERCISE 1** 

Implement a Kripke Structure for the following Python program and check some formulas:
```
0: V=[4,2,5,6,8]

1: max=0

2: for i in range(size(V)):

3:    if V[i]>max:

4:      max=V[i]

5: 
```

In [32]:
#TYPE YOUR IMPLEMENTATION HERE

K = Kripke(
    S=[0, 1, 2, 3, 4], 
    S0=[0],
    R=[(0, 1), (1, 2), (2, 3), (3, 4), (4, 2), (4, 5), (2, 5), (5, 5)],
    L={
        0: set(["v_arr_4,2,5,6,8"]), 
        1: set(["v_arr_4,2,5,6,8", "max_igual_0"]), 
        2: set(["v_arr_4,2,5,6,8", "max_igual_0"]),
        3: set(["v_arr_4,2,5,6,8", "max_igual_0"]),
        4: set(["v_arr_4,2,5,6,8", "v[i]_maior_que_max", "max_igual_v[i]"]),
    }
    # no estado 1 poderia ser x_igual_a_5
)

**EXERCISE 2** 

Implement a Kripke Structure for the following Python program and check some formulas:

V=[4,2,5,6,8]

max=0

for i in range(size(V)-1):

    for j in range(size(V)):

      if V[j]<V[i]:

        aux = V[i]

        V[i]=V[j]

        V[j]=aux


    

In [ ]:
#TYPE YOUR SOLUTION HERE